In [5]:
from subprocess import Popen
import subprocess
import os
# import matplotlib.pyplot as plt
import pyecharts.options as opts
from pyecharts.charts import Pie
from snapshot_phantomjs import snapshot
from pyecharts.render import make_snapshot

# abspath = input("nova-like-timing-file:")
# abspath = "/home/deadpool/HUNTER-REPO/tests/killer-study/FIG-MetaBreakDown/M_DATA/fio/randwrite copy/KILLER"
abspath = "/home/deadpool/HUNTER-REPO/tests/killer-study/FIG-MetaBreakDown/M_DATA/fio/write copy/KILLER-NAIVE"
# abspath = "/home/deadpool/HUNTER-REPO/tests/killer-study/FIG-MetaBreakDown/M_DATA/fio/write copy/KILLER"
def extract_value(line, pos: int):
    # using awk to extract the value
    cmd = "echo " + '"' + line + '"' + " | awk '{print $" + str(pos) + "}'"
    ret = Popen(cmd, shell=True, stdout=subprocess.PIPE)
    return ret.stdout.read().decode("utf-8").strip()
name_filters = ["mount", "new_init", "write", "dax_read"] 
timers = []
with open(abspath, "r") as f:
    lines = f.readlines()
    for line in lines:
        if line.count(":") != 0:
            line = line.strip("\n")
            timer_name = extract_value(line, 1).split(":")[0]
            # nsW
            timer_value = int(extract_value(line, 5).split(",")[0])
            # s
            timer_value_rounded = round(timer_value / 1000 / 1000 / 1000, 3)
            if timer_value_rounded > 0 and timer_name not in name_filters:
                timers.append((timer_name, timer_value_rounded))

pie = Pie(init_opts=opts.InitOpts(js_host="./"))
pie.add("", timers, center=["40%", "50%"]).set_global_opts(
    legend_opts=opts.LegendOpts(type_='scroll', pos_left='80%', orient='vertical'),
    title_opts= opts.TitleOpts(title="Nova-like-timing-breakdown")).set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))

make_snapshot(snapshot, pie.render(), "nova-like-timing-breakdown.pdf", is_remove_html=True)
pie.render_notebook()
